# Importação da biblioteca e base de dados

In [0]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from IPython.display import display


#1. Carregar e Preparar os Dados

In [0]:
# Carregar o DataFrame a partir da tabela Spark e transformar em pandas
df_spark = spark.table('workspace.gold.dim_dados_clientes_cluster')
df = df_spark.toPandas()
df

,cod_cliente,uf,cidade,qtd_contratos_distintos_ativos,qtd_contratos_ativos,pais,situacao_contrato,nome_produto,marca_totvs,dt_assinatura_contrato,segmento_cliente,cliente_desde,faixa_faturamento,hospedagem,qtd_contratacoes_12m,qtd_total_contratos,qtd_marca_distinta,faixa_fat
0,99958,SC,JOINVILLE,NaN,NaN,105,GRATUITO,SMS FULL TOTVS TRAD,CROSS - TRADICIONAL,2016-04-07,SERVICOS,2016-04-07,FAIXA 09 - DE 300 M ATE 500 M,None,NaN,8,3.0,400
1,T00053,RJ,RIODEJANEIRO,30.0,44.0,105,ATIVO,SMS COLAB NEO 2500 DOC,MANUFATURA - PARCEIRO,2015-02-27,MANUFATURA,1992-02-26,FAIXA 05 - DE 35 M ATE 50 M,ON PREMISES,1.0,80,11.0,42
2,T00053,RJ,RIODEJANEIRO,30.0,44.0,105,CANCELADO,HORA SUPORTE,SERVICOS DE IMPLANTACAO,1997-11-28,MANUFATURA,1992-02-26,FAIXA 05 - DE 35 M ATE 50 M,ON PREMISES,1.0,80,11.0,42
3,99958,SC,JOINVILLE,NaN,NaN,105,GRATUITO,CDU FULL TOTVS TRAD,CROSS - TRADICIONAL,2016-04-07,SERVICOS,2016-04-07,FAIXA 09 - DE 300 M ATE 500 M,None,NaN,8,3.0,400
4,T00053,RJ,RIODEJANEIRO,30.0,44.0,105,TROCADO,PROGRESS USER 11 CDU,PROGRESS,2017-11-22,MANUFATURA,1992-02-26,FAIXA 05 - DE 35 M ATE 50 M,ON PREMISES,1.0,80,11.0,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238592,TFEEYP,RS,CANELA,13.0,13.0,105,GRATUITO,EMISSOR FISCAL NFC-E,HOSPITALIDADE - CORE,2025-03-24,HOSPITALITY,2025-03-24,SEM INFORMAÇÕES DE FATURAMENTO,ON PREMISES,1.0,15,1.0,0
238593,TFEEYP,RS,CANELA,13.0,13.0,105,GRATUITO,HIGS - INTEGRACOES,HOSPITALIDADE - CORE,2025-03-24,HOSPITALITY,2025-03-24,SEM INFORMAÇÕES DE FATURAMENTO,ON PREMISES,1.0,15,1.0,0
238594,TFEEYQ,SP,SAOPAULO,12.0,12.0,105,ATIVO,CLOUD EXCEDENTE TRAD,CLOUD,2025-03-24,JURIDICO,2025-03-24,SEM INFORMAÇÕES DE FATURAMENTO,ON PREMISES,1.0,12,3.0,0
238595,TFEEYQ,SP,SAOPAULO,12.0,12.0,105,ATIVO,IAAS-DDADO-50GB-2KIOPS-LEVE-M,CLOUD,2025-03-24,JURIDICO,2025-03-24,SEM INFORMAÇÕES DE FATURAMENTO,ON PREMISES,1.0,12,3.0,0


In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from collections import defaultdict

# Carregar o DataFrame a partir da tabela Spark e transformar em pandas
df_spark = spark.table('workspace.gold.dim_dados_clientes_cluster')
df = df_spark.toPandas()

# 1. TRATAMENTO DA VARIÁVEL DE FATURAMENTO
# Definindo o dicionário de mapeamento para a variável ordinal de faturamento
mapeamento_faixas = {
    'FAIXA 00 - ATE 4,5 M': 0,
    'FAIXA 01 - DE 4,5 M ATE 7,5 M': 1,
    'FAIXA 02 - DE 7,5 M ATE 15 M': 2,
    'FAIXA 03 - DE 15 M ATE 25 M': 3,
    'FAIXA 04 - DE 25 M ATE 35 M': 4,
    'FAIXA 05 - DE 35 M ATE 50 M': 5,
    'FAIXA 06 - DE 50 M ATE 75 M': 6,
    'FAIXA 07 - DE 75 M ATE 150 M': 7,
    'FAIXA 08 - DE 150 M ATE 300 M': 8,
    'FAIXA 09 - DE 300 M ATE 500 M': 9,
    'FAIXA 10 - DE 500 M ATE 850 M': 10,
    'FAIXA 11 - ACIMA DE 850 M': 11,
    'SEM INFORMAÇÕES DE FATURAMENTO': -1
}

# Criando as novas colunas no DataFrame
df['faturamento_medio_ordinal'] = df['faixa_faturamento'].map(mapeamento_faixas)
df['faturamento_medio_ausente'] = (df['faixa_faturamento'] == 'SEM INFORMAÇÕES DE FATURAMENTO').astype(int)

# 2. ENGENHARIA DE FEATURES DE PRODUTO
# Mapeamento de produtos para categorias de negócio
mapeamento_produtos = defaultdict(lambda: 'PRODUTO_OUTROS')

# Soluções de RH e Ponto
produtos_rh = ['AHGORA', 'MEU RH', 'CLOCKIN', 'FEEDZ', 'PONTO', 'RH']
for p in produtos_rh: mapeamento_produtos[p] = 'PRODUTO_RH'

# Soluções de Gestão/ERP
produtos_gestao = ['TOTVS', 'CDU', 'SMS', 'LGT', 'PROTHEUS', 'INTERA', 'ERP', 'SIGA', 'DATANAVIEWER', 'DATASUL', 'EMS', 'SISTEMA PONTO']
for p in produtos_gestao: mapeamento_produtos[p] = 'PRODUTO_GESTAO'

# Soluções de Infraestrutura/Cloud
produtos_infra = ['CLOUD', 'TEC', 'PROCESSOR', 'IAAS', 'PROGRESS', 'SQL', 'ORACLE', 'SERVIDOR']
for p in produtos_infra: mapeamento_produtos[p] = 'PRODUTO_INFRA'

# Soluções de Varejo/Comércio
produtos_comercial = ['CONSINCO', 'MISTERCHEF', 'BEMATECH', 'TEF', 'PDV', 'NFC-E', 'VAREJO', 'COMERCIO', 'HOMECENTER']
for p in produtos_comercial: mapeamento_produtos[p] = 'PRODUTO_COMERCIAL'

# Soluções Financeiras e Fiscais
produtos_financeiros = ['FINANCEIRO', 'FISCAL', 'CONTABIL', 'FATURAMENTO', 'REINF', 'SPED', 'TSS']
for p in produtos_financeiros: mapeamento_produtos[p] = 'PRODUTO_FINANCEIRO'

# Mapear cada nome_produto para uma categoria
df['categoria_produto'] = df['nome_produto'].str.split(' ').str[0].map(mapeamento_produtos)

# Agrupar por cliente e contar a ocorrência de cada categoria
df_produto_agregado = df.groupby('cod_cliente')['categoria_produto'].value_counts().unstack(fill_value=0)

# Juntar as novas features de volta ao DataFrame principal
df = df.merge(df_produto_agregado, on='cod_cliente', how='left').fillna(0)

# O seu DataFrame 'df' agora está pronto para a etapa de clusterização
print(df.head())
print(df.columns)

  cod_cliente  uf        cidade  ...  PRODUTO_INFRA  PRODUTO_OUTROS  PRODUTO_RH
0       99958  SC     JOINVILLE  ...              0               1           0
1      T00053  RJ  RIODEJANEIRO  ...              7              35           0
2      T00053  RJ  RIODEJANEIRO  ...              7              35           0
3       99958  SC     JOINVILLE  ...              0               1           0
4      T00053  RJ  RIODEJANEIRO  ...              7              35           0

[5 rows x 27 columns]
Index(['cod_cliente', 'uf', 'cidade', 'qtd_contratos_distintos_ativos',
       'qtd_contratos_ativos', 'pais', 'situacao_contrato', 'nome_produto',
       'marca_totvs', 'dt_assinatura_contrato', 'segmento_cliente',
       'cliente_desde', 'faixa_faturamento', 'hospedagem',
       'qtd_contratacoes_12m', 'qtd_total_contratos', 'qtd_marca_distinta',
       'faixa_fat', 'faturamento_medio_ordinal', 'faturamento_medio_ausente',
       'categoria_produto', 'PRODUTO_COMERCIAL', 'PRODUTO_FINANCEIRO

In [0]:
# Salvar a coluna 'cod_cliente' antes de dropar para usar no DataFrame final
cod_cliente = df['cod_cliente']

In [0]:
# Remover colunas que não serão utilizadas para a clusterização
df = df.drop(columns=[
    'cod_cliente', 'hospedagem',
    'cliente_desde', 'marca_totvs'
])

# 2. Encoding de Variáveis Categóricas

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238597 entries, 0 to 238596
Data columns (total 23 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   uf                              238597 non-null  object 
 1   cidade                          238597 non-null  object 
 2   qtd_contratos_distintos_ativos  238597 non-null  float64
 3   qtd_contratos_ativos            238597 non-null  float64
 4   pais                            238597 non-null  int32  
 5   situacao_contrato               238597 non-null  object 
 6   nome_produto                    238597 non-null  object 
 7   dt_assinatura_contrato          238597 non-null  object 
 8   segmento_cliente                238597 non-null  object 
 9   faixa_faturamento               238597 non-null  object 
 10  qtd_contratacoes_12m            238597 non-null  float64
 11  qtd_total_contratos             238597 non-null  int32  
 12  qtd_marca_distin

In [0]:
# Criar o DataFrame final com todas as variáveis processadas
dados_clientes = df.drop(columns=['uf','situacao_contrato','cidade','pais','faixa_fat', 'segmento_cliente','nome_produto','faturamento_medio_ausente', 'categoria_produto', 'uf','faixa_faturamento', 'qtd_contratos_ativos','qtd_contratacoes_12m', 'qtd_total_contratos'])

In [0]:
# Tratar valores NaN restantes (se houverem, geralmente da parte numérica)
dados_clientes = dados_clientes.fillna(0)

In [0]:
dados_clientes["qtd_marca_distinta"] = dados_clientes["qtd_marca_distinta"].astype(int)
spark_df = spark.createDataFrame(dados_clientes)
spark_df.createOrReplaceTempView("dados_clientes")

In [0]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, MiniBatchKMeans, SpectralClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
import numpy as np

# --- Preparação dos dados


# 1. Criar a nova feature de contagem de produtos
features_binarias = [
    'PRODUTO_COMERCIAL', 'PRODUTO_FINANCEIRO', 'PRODUTO_GESTAO',
    'PRODUTO_INFRA', 'PRODUTO_OUTROS', 'PRODUTO_RH'
]
features_binarias_existentes = [f for f in features_binarias if f in dados_clientes.columns]
dados_clientes['qtd_total_produtos'] = dados_clientes[features_binarias_existentes].sum(axis=1)

# 2. Selecionar e normalizar as features
features_para_clusterizar = [
    'qtd_marca_distinta',
    'tempo_cliente_em_meses',
    'faturamento_medio_ordinal',
    'qtd_total_produtos'
]
features_para_clusterizar_existentes = [f for f in features_para_clusterizar if f in dados_clientes.columns]
X = dados_clientes[features_para_clusterizar_existentes]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# --- Executar e Avaliar os Modelos ---

# K-Means
print("="*50)
print("Análise K-Means")
kmeans = KMeans(n_clusters=4, random_state=45, n_init='auto')
dados_clientes['cluster_kmeans'] = kmeans.fit_predict(X_scaled)
silhouette_kmeans = silhouette_score(X_scaled, dados_clientes['cluster_kmeans'])
print(f"Silhueta K-Means: {silhouette_kmeans:.4f}")
print("Distribuição dos Clusters:")
print(dados_clientes['cluster_kmeans'].value_counts().sort_index())


# GMM
print("\n"+"="*50)
print("Análise Gaussian Mixture Model (GMM)")
gmm = GaussianMixture(n_components=4, random_state=45)
gmm.fit(X_scaled)
dados_clientes['cluster_gmm'] = gmm.predict(X_scaled)
silhouette_gmm = silhouette_score(X_scaled, dados_clientes['cluster_gmm'])
print(f"Silhueta GMM: {silhouette_gmm:.4f}")
print("Distribuição dos Clusters:")
print(dados_clientes['cluster_gmm'].value_counts().sort_index())


# MiniBatchKMeans
print("\n"+"="*50)
print("Análise MiniBatchKMeans")
mb_kmeans = MiniBatchKMeans(n_clusters=4, random_state=45, n_init='auto', batch_size=256)
dados_clientes['cluster_mb_kmeans'] = mb_kmeans.fit_predict(X_scaled)
silhouette_mb_kmeans = silhouette_score(X_scaled, dados_clientes['cluster_mb_kmeans'])
print(f"Silhueta MiniBatchKMeans: {silhouette_mb_kmeans:.4f}")
print("Distribuição dos Clusters:")
print(dados_clientes['cluster_mb_kmeans'].value_counts().sort_index())


# Spectral Clustering
print("\n"+"="*50)
print("Análise Spectral Clustering")
spectral = SpectralClustering(n_clusters=4, random_state=45, affinity='nearest_neighbors')
dados_clientes['cluster_spectral'] = spectral.fit_predict(X_scaled)
silhouette_spectral = silhouette_score(X_scaled, dados_clientes['cluster_spectral'])
print(f"Silhueta Spectral Clustering: {silhouette_spectral:.4f}")
print("Distribuição dos Clusters:")
print(dados_clientes['cluster_spectral'].value_counts().sort_index())


# DBSCAN
print("\n"+"="*50)
print("Análise DBSCAN")
# DBSCAN não exige 'n_clusters', mas 'eps' e 'min_samples' precisam ser ajustados.
# Para este teste, vamos usar valores padrão.
dbscan = DBSCAN(eps=0.5, min_samples=5)
dados_clientes['cluster_dbscan'] = dbscan.fit_predict(X_scaled)
# Checar se mais de um cluster foi encontrado antes de calcular a silhueta
if len(np.unique(dados_clientes['cluster_dbscan'])) > 1:
    silhouette_dbscan = silhouette_score(X_scaled, dados_clientes['cluster_dbscan'])
    print(f"Silhueta DBSCAN: {silhouette_dbscan:.4f}")
else:
    print("Silhueta: Não foi possível calcular (apenas um cluster ou todos são ruído).")
print("Distribuição dos Clusters:")
print(dados_clientes['cluster_dbscan'].value_counts().sort_index())

---------------------------------------------------------------------------
The Python process exited with exit code 137 (SIGKILL: Killed). This may have been caused by an OOM error. Check your command's memory usage.



The last 10 KB of the process's stderr and stdout can be found below. See driver logs for full logs.
---------------------------------------------------------------------------
Last messages on stderr:
^^
AttributeError: 'NoneType' object has no attribute 'split'
/databricks/python/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0xff5296d9f420>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    se

In [0]:
# Vamos incluir todas as features numéricas que usamos para o modelo
features_para_analise = [
    'faturamento_medio_ordinal',
    'faturamento_medio_ausente',
    'tempo_cliente_em_meses',
    'qtd_contrato_cancelado',
    'qtd_marca_distinta',
    'PRODUTO_RH',
    'PRODUTO_GESTAO',
    'PRODUTO_INFRA',
    'PRODUTO_COMERCIAL',
    'PRODUTO_FINANCEIRO',
    'OUTROS'
]


# Agrupa o DataFrame pela coluna de cluster e calcule a média de cada feature
perfil_clusters = dados_clientes.groupby('cluster_gmm')[features_para_analise].mean()


# É útil saber quantos clientes estão em cada grupo para a análise
contagem_clientes = dados_clientes['cluster_gmm'].value_counts().sort_index()
perfil_clusters['Contagem de Clientes'] = contagem_clientes


# Reordena as colunas para que a contagem apareça primeiro
perfil_clusters = perfil_clusters.sort_values('Contagem de Clientes', ascending=False)
perfil_clusters = perfil_clusters[['Contagem de Clientes'] + [col for col in perfil_clusters.columns if col != 'Contagem de Clientes']]

display(perfil_clusters)

,Contagem de Clientes,faturamento_medio_ordinal,faturamento_medio_ausente,tempo_cliente_em_meses,qtd_contrato_cancelado,qtd_marca_distinta,PRODUTO_RH,PRODUTO_GESTAO,PRODUTO_INFRA,PRODUTO_COMERCIAL,PRODUTO_FINANCEIRO,OUTROS
cluster_gmm,,,,,,,,,,,,
0,3784,-1.00,1.00,41.65,0.00,1.10,0.44,0.11,0.00,0.09,0.00,0.36
2,2869,1.48,0.00,119.72,1.31,3.18,0.01,0.45,0.02,0.07,0.00,0.44
1,2688,4.97,0.00,187.26,16.57,5.72,0.01,0.49,0.05,0.02,0.00,0.43
3,1274,-1.00,1.00,91.79,6.62,2.43,0.05,0.26,0.02,0.11,0.01,0.55
